In [339]:
import numpy as np

In [340]:
def init(ns,num_states,inp_dims):
    # initial cluster responsibilities
    qs = np.ones((ns,num_states)) / num_states
    mus = np.random.randn(num_states,inp_dims)
    # initial cluster probabolities
    pis = np.ones((num_states,)) / num_states
    sigmas = np.random.randn(num_states,inp_dims,inp_dims)
    return qs, pis, mus, sigmas

def E_step(ns,_qs,pis,mus,sigmas,xs):
    qs = np.zeros(_qs.shape)
    num_states = mus.shape[0]
    inp_dims = len(xs[0])
    for n in range(ns):
        # for each n in dataset - see mathematical derivation in probability.pdf:
        for k in range(num_states):
            products = np.einsum('i,ij,j->',(xs[n]-mus[k]),np.linalg.inv(sigmas[k]),(xs[n]-mus[k]))
            u = pis[k]/(((2*np.pi)**inp_dims*abs(np.linalg.det(sigmas[k])))**(1/2)) * np.exp(-0.5*products)
            #!! the determinant is sometimes -ve causing error in the square root - absolute here is dodgy, NOT A FIX
            qs[n][k] = u
        qs[n] /= np.sum(qs[n])
    return qs

def M_step(ns,qs,xs,_mus):
    num_states = _mus.shape[0]
    inp_dims = _mus.shape[1]
    N_ks = np.sum(qs, axis=0)
    mus = np.zeros(_mus.shape)
    sigmas = np.zeros((num_states,inp_dims,inp_dims))
    pis = np.zeros((num_states,))
    for k in range(len(N_ks)): 
        mus[k] = (1/N_ks[k]) * np.einsum('i,ij->j',qs.T[k],xs) # sum over N removes dependence on N, only on k now
        mus_stack = np.stack([mus for n in range(ns)],axis=1) # (KxNxD): for use of einsum without loop over n below:
        sum_product_sigmas = np.einsum('j,ji,jk->ik',qs.T[k],(xs-mus_stack[k]),(xs-mus_stack[k]))
        sigmas[k] = (1/N_ks[k]) * sum_product_sigmas
        pis[k] = N_ks[k] / ns
    return mus, sigmas, pis

In [341]:
def run(iterations,xs):
    xs = np.array(xs)
    ns = xs.shape[0] # number of input datapoints
    inp_dims = xs.shape[1] # dimension of the input space implicit from inputs
    num_states = 2 # number of clusters to fit to
    _qs,_pis,_mus,_sigmas = init(ns,num_states,inp_dims)
    # Initially randomly allocate the datapoints to clusters (display purposes only):
    memberships = [np.random.randint(0,num_states,size=(ns,))]
    param_history = [_mus,_sigmas,_pis]
    for i in range(iterations):
        qs = E_step(ns,_qs,_pis,_mus,_sigmas,xs)
        # As free energy minimised when KL divergence is 0 between q and p(s|X,theta), we know that the memberships can be approximated by the optimal qs:
        memberships.append(np.argmax(qs, axis=1))
        mus,sigmas,pis = M_step(ns,qs,xs,_mus)
        _qs = qs
        _pis = pis
        _mus = mus
        _sigmas = sigmas
        param_history.append([_mus,_sigmas,_pis])
    return np.array(memberships,dtype=np.float32), (param_history) #  a history (iterations x N) of the memberships of each datapoint

In [342]:
x = []
truth = np.zeros(100)
for i in range(100):
    r = np.random.randint(0,2)
    if r == 0:
        x.append(np.random.normal(0.5,0.1,size=(2,)))
        truth[i] = 0
    else: 
        x.append(np.random.normal(-0.5,0.1,size=(2,)))
        truth[i] = 1
xs = np.array(x)

model = run(10,xs)

print(model[0][-1])
print()
print(truth)
print()
print(*model[1][-1])

[1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1.
 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0.
 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1.
 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0.
 0. 0. 1. 1.]

[0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0.
 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.
 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.
 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1.
 1. 1. 0. 0.]

[[-0.513843   -0.50001549]
 [ 0.49760483  0.49571426]] [[[ 0.01545109  0.00191352]
  [ 0.00191352  0.01471601]]

 [[ 0.0087762  -0.00050298]
  [-0.00050298  0.01061639]]] [0.36 0.64]
